In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf
from tensorflow import keras

In [2]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

In [3]:
tf.keras.utils.get_file('flower_photos',origin=dataset_url,cache_dir='.',untar=True)

'./datasets/flower_photos'

In [4]:
import pathlib

In [5]:
data_dir = './datasets/flower_photos'

In [6]:
data_dir = pathlib.Path(data_dir)

In [7]:
data_dir

PosixPath('datasets/flower_photos')

In [8]:
roses = list(data_dir.glob('roses/*'))

In [9]:
flower_images_dict = {
    'roses':list(data_dir.glob('roses/*')),
    'daisy':list(data_dir.glob('daisy/*')),
    'dandelion':list(data_dir.glob('dandelion/*')),
    'sunflowers':list(data_dir.glob('sunflowers/*')),
    'tulips':list(data_dir.glob('tulips/*')),
}

In [10]:
flower_labels_dict = {
    'roses':0,
    'daisy':1,
    'dandelion':2,
    'sunflowers':3,
    'tulips':4,
}

In [11]:
img = cv2.imread(str(flower_images_dict['roses'][0]))

In [12]:
X ,y = [],[]
for flower_name,images in flower_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(180,180))
        X.append(resized_img)
        y.append(flower_labels_dict[flower_name])

In [13]:
X = np.array(X)
y = np.array(y)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [15]:
X_train = X_train/255
X_test = X_test/255

In [16]:
model = keras.Sequential([
  keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(5)
])

In [17]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [18]:
model.fit(X_train,y_train,epochs=5)

Epoch 1/5
86/86 [==============================] - 26s 304ms/step - loss: 1.4043 - accuracy: 0.4026
Epoch 2/5
86/86 [==============================] - 26s 302ms/step - loss: 1.0478 - accuracy: 0.5825
Epoch 3/5
86/86 [==============================] - 26s 299ms/step - loss: 0.8163 - accuracy: 0.6886
Epoch 4/5
86/86 [==============================] - 26s 301ms/step - loss: 0.6184 - accuracy: 0.7758
Epoch 5/5
86/86 [==============================] - 26s 300ms/step - loss: 0.3988 - accuracy: 0.8619


In [19]:
model.evaluate(X_test,y_test)

29/29 [==============================] - 2s 56ms/step - loss: 1.3637 - accuracy: 0.6035


[1.3636800050735474, 0.6034858226776123]

In [20]:
data_augmentation = keras.Sequential(
  [
    keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(180, 
                                                              180,
                                                              3)),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
    keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [21]:
num_classes = 5

model = keras.Sequential([
    data_augmentation,
  keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Dropout(0.2),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(num_classes)
])

In [22]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [23]:
model.fit(X_train,y_train,epochs=5)

Epoch 1/5
86/86 [==============================] - 29s 338ms/step - loss: 1.4765 - accuracy: 0.3837
Epoch 2/5
86/86 [==============================] - 29s 337ms/step - loss: 1.0907 - accuracy: 0.5672
Epoch 3/5
86/86 [==============================] - 29s 338ms/step - loss: 0.9811 - accuracy: 0.6112
Epoch 4/5
86/86 [==============================] - 29s 336ms/step - loss: 0.9141 - accuracy: 0.6508
Epoch 5/5
86/86 [==============================] - 29s 335ms/step - loss: 0.8559 - accuracy: 0.6755


In [24]:
model.evaluate(X_test,y_test)

29/29 [==============================] - 2s 57ms/step - loss: 0.9332 - accuracy: 0.6209


[0.9332038760185242, 0.6209150552749634]